# Scrape Amazon Reviews

Scraping Amazon Dog Collar reviews for a friend who would like some analysis on what factors might be driving negative reviews.  Will need to install docker to use splash to scrape amazon.  

### Imports

In [1]:
import requests
from requests import get
import pandas as pd
from bs4 import BeautifulSoup

### URL to Scrape

In [5]:
# stash away the url in a variable
url = "https://www.amazon.com/Lifetime-Walking-American-Pitbull-Shepherd/product-reviews/B087NQ9V3K/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1"
reviewlist = []

### Docker workaround using splash

In [7]:
# use splash to past the 503 response you would get from trying to scrape without it
r = requests.get('http://localhost:8050/render.html', params={'url': url, 'wait': 2})


In [8]:
# make the review soup
soup = BeautifulSoup(r.text, 'html.parser')

In [9]:
# check out the title 
print(soup.title.text)

Amazon.com: Customer reviews: W/W Lifetime Gold Dog Chain Collar Walking Metal Chain Collar with Design Secure Buckle,18K Cuban Link Strong Heavy Duty Chew Proof for Medium Dogs(19MM, 15")


In [10]:
# find all returns in a list that we can loop through
reviews = soup.find_all('div', {'data-hook': 'review'})

In [19]:
# create loop to grab title, rating, and review
for item in reviews:
    # make into a dictionary by turning title, rating, and body into keys and data that we are
    # scraping into values
    review = {
    'product': soup.title.text.replace('Amazon.com: Customer reviews:', '').strip(),
    'title' : item.find('a', {'data-hook': 'review-title'}).text.strip(),
    'rating' : float(item.find('i', {'data-hook': 'review-star-rating'}).text.replace('out of 5 stars', '').strip()),
    'review_body' : item.find('span', {'data-hook': 'review-body'}).text.strip(),}
    
    
    

### Make everything into useful functions

In [2]:
def get_soup(url):
    r = requests.get('http://localhost:8050/render.html', params={'url': url, 'wait': 2})
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

In [3]:
reviewlist = []

In [6]:
def get_reviews(soup):
    reviews = soup.find_all('div', {'data-hook': 'review'})
    try:
        for item in reviews:
            review = {
            'product': soup.title.text.replace('Amazon.com: Customer reviews:', '').strip(),
            'title' : item.find('a', {'data-hook': 'review-title'}).text.strip(),
            'rating' : float(item.find('i', {'data-hook': 'review-star-rating'}).text.replace('out of 5 stars', '').strip()),
            'review_body' : item.find('span', {'data-hook': 'review-body'}).text.strip(),
            }
            reviewlist.append(review)
    except:
        pass

In [7]:
soup = get_soup('https://www.amazon.com/Lifetime-Walking-American-Pitbull-Shepherd/product-reviews/B087NQ9V3K/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1')

In [8]:
get_reviews(soup)

In [9]:
print(len(reviewlist))

10


In [10]:
print(reviewlist[0])

{'product': 'W/W Lifetime Gold Dog Chain Collar Walking Metal Chain Collar with Design Secure Buckle,18K Cuban Link Strong Heavy Duty Chew Proof for Medium Dogs(19MM, 15")', 'title': 'So cute!!!', 'rating': 5.0, 'review_body': 'So cute! It’s heavier duty than I was anticipating, but does feel a little cheap. I wouldn’t trust attaching a leash to it, but I don’t think that’s the point of something like this. Ordering another one for my older Vizsla now!'}


### Deal with pages

### Check Response
This was the response before using splash

In [3]:
response = get(url)

In [4]:
response

<Response [503]>

Apparently Amazon does not like scrapers.  This is not the response we want, so a work around is needed.  